In order to run this notebook, you need to install the following packages:

- `requests`
- `tqdm`
- `matplotlib`
- `scipy`
- `xarray-leaflet`

The recommended way is: `conda install -c conda-forge requests tqdm matplotlib scipy xarray_leaflet`

In [ ]:
import requests
import warnings
import os
from tqdm import tqdm
import zipfile
import rioxarray
import xarray as xr
import numpy as np
import scipy.ndimage
import xarray_leaflet
from rasterio.warp import Resampling
from xarray_leaflet.transform import passthrough, normalize
from ipyleaflet import Map, basemaps

We first download the awesome [HydroSHEDS](https://hydrosheds.org) dataset, and in particular the flow accumulation for South America. You can think of flow accumulation as a potential river flow, so we will have a visual representation of rivers, including the great Amazon river.

In [ ]:
url = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/sa_acc_30s_grid.zip'
filename = os.path.basename(url)
name = filename[:filename.find('_grid')]
adffile = name + '/' + name + '/w001001.adf'

if not os.path.exists(adffile):
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024) + 1):
            if chunk:
                f.write(chunk)
                f.flush()
    zip = zipfile.ZipFile(filename)
    zip.extractall('.')

Let's open the box and see what's in there.

In [ ]:
da = rioxarray.open_rasterio(adffile, masked=True)
da

We just need to select the band.

In [ ]:
da = da.sel(band=1)

`xarray-leaflet` has two modes, one is for static maps and the other for dynamic maps. By dynamic we mean that you won't see the same thing depending on the current map view. For instance, if you zoom in you might want to show details that would otherwise be hidden. So the map really adapts to where you look on the Earth. The downside of dynamic maps is that you will see more flickering as you interact with the map, because tiles have to be refreshed as soon as you drag or zoom.

On the contrary, static maps won't be as reactive with respect to the location. Of course they will refine as you zoom in, but they won't change dramatically. On the other hand, they are rendered really smoothly.

This example can be configured to work both in static or dynamic mode. You can set the following `dynamic` variable to `True` or `False` in order to switch between the two modes (you will have to re-execute the cells to take it into account).

In [ ]:
# you can change the value of this variable to True or False
# and see the difference in the interaction with the map

dynamic = True

There are 4 stages in xarray-leaflet's data pipe where you can apply transformations to your data. The first one operates on the whole `DataArray`, even the part that is not visible on the map. This is often where you want to normalize your data to keep the values between 0 and 1. The default transformation for this stage does exactly that, so if you don't want to add any transformation at this stage, just don't redefine it. In this example, we will keep the default transformation for the static mode, and have our own normalization in the next stage for the dynamic mode.

The second transformation operates on the visible part. Often, this is where you want to apply dynamic transformations. Transformations are Python functions accepting the data as first argument and optional parameters from the previous transformation. In this example, we won't change the data for the static mode, and normalize it for the dynamic mode.

In [ ]:
if dynamic:
    # dynamic map
    transform0 = passthrough
    transform1 = normalize
else:
    # static map
    transform0 = normalize
    transform1 = passthrough

The third transformation applies to the data contained in each Leaflet tile before reprojection. Reprojection needs your data to fit into memory, so you may want to downsample your data and keep approximately the same number of points as there are in a tile (256 x 256). The default transformation for this stage does just that, but we will change it a little bit so that the aggregation function is not `mean` but `max`. We also downsample our data but it is very specific to this dataset.

In [ ]:
def transform2(array, *args, **kwargs):
    tile_width = kwargs['tile_width']
    tile_height = kwargs['tile_height']
    ny, nx = array.shape
    wx = nx // (tile_width // 2)
    wy = ny // (tile_height // 2)
    dim = {}
    if wx > 1:
        dim['x'] = wx
    if wy > 1:
        dim['y'] = wy
    array = array.coarsen(**dim, boundary='pad')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        array = xr.core.rolling.DataArrayCoarsen.max(array)
    return array

In the fourth and last transformation, the data has already been reprojected to Web Mercator (the projection used in Leaflet by default). This is the last opportunity to transform your data before it is saved to a `PNG` file and sent to the browser. That's often when you want to apply styling. Here we make the rivers appear thicker and we reduce the amplitude with a square root function.

In [ ]:
def transform3(array):
    radius = 2
    circle = np.zeros((2*radius+1, 2*radius+1)).astype('uint8')
    y, x = np.ogrid[-radius:radius+1,-radius:radius+1]
    index = x**2 + y**2 <= radius**2
    circle[index] = 1
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        array = np.sqrt(array)
    array = scipy.ndimage.maximum_filter(array, footprint=circle)
    return array

We are almost done, we just need to create a map before passing it to our `DataArray` extension.

In [ ]:
m = Map(center=[-20, -60], zoom=3, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m

To show our data on the map, we call `leaflet.plot()` on our `DataArray`, and pass as parameters the map, the transformation functions, the `dynamic` value, and the resampling method for the reprojection.

In [ ]:
l = da.leaflet.plot(m,
                    transform0=transform0,
                    transform1=transform1,
                    transform2=transform2,
                    transform3=transform3,
                    dynamic=dynamic,
                    resampling=Resampling.max)
l.interact(opacity=(0.0,1.0,0.1))

You can now drag and zoom, explore different locations, and if you chose `dynamic=True` you should see details being revealed as you interact. If you chose `dynamic=False`, the map should just refine as you zoom in, but should be more reactive and less flickering. 